In [49]:
import pandas as pd
import json
from datetime import datetime
import holidays
import os
from datetime import timedelta
import numpy as np

In [9]:
#df filtered by houston, needed to filter by state = TX (NOT IMPLEMENTED)

subway = pd.read_csv('/Users/mazcu/Downloads/Subway_Houston_days.csv', encoding="utf-8")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7,8,9,17,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
def is_weekend(df):
    df['date'] = pd.to_datetime(df['date']) 

    df['is_weekend'] = df['date'].dt.day_name()
    df.loc[df['is_weekend'] == "Saturday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Sunday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Monday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Tuesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Wednesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Thursday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Friday", 'is_weekend'] = 0

    return df

def is_holiday(df):
    holi = holidays.CountryHoliday('US', prov="Houston", state='TX')
    df["is_holiday"] = [1 if d in holi else 0 for d in df["date"]]
    return df

def last_day(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 1)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')
        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day in yesterday:
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
        
        new['visits_last_day'] = l

        if i == 0:
            old = new
            i += 1
        else:
            old = pd.concat([old, new])
    return old

def last_week(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 7)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')
        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day in yesterday:
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
        
        new['visits_last_week'] = l

        if i == 0:
            old = new
            i += 1
        else:
            old = pd.concat([old, new])
    return old

subway = is_weekend(subway)
subway = is_holiday(subway)
subway = last_day(subway)
subway = last_week(subway)


<ipython-input-10-f7b8fb7eba20>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['visits_last_day'] = l
<ipython-input-10-f7b8fb7eba20>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['visits_last_week'] = l


In [11]:
subway['date'].count

<bound method Series.count of 3441    2020-01-01
3442    2020-01-02
3443    2020-01-03
3444    2020-01-04
3445    2020-01-05
           ...    
90590   2021-03-27
90591   2021-03-28
90592   2021-03-29
90593   2021-03-30
90594   2021-03-31
Name: date, Length: 92796, dtype: datetime64[ns]>

import os

for filename in os.listdir('/Users/mazcu/Downloads/safegraph_open_census_data/data/'): 
    print(filename)
    cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/'+filename, encoding="utf-8")
    try:
        print(filename, cosa['B19013e1'])
    except:
        pass

In [29]:
cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/cbg_b19.csv', encoding="utf-8", dtype = {"census_block_group": "category"})


In [13]:
df = subway

df['poi_cbg'] = df['poi_cbg'].map(int)
df['poi_cbg'] = df['poi_cbg'].map(str)


In [18]:
new = df[df['region'] == 'TX']
cbgs = set(new['poi_cbg'])

In [31]:
cosa = cosa[cosa['census_block_group'].isin(cbgs)]
cosa = cosa[['census_block_group', 'B19013e1']]

In [54]:
"DO NOT EXECUTE, MAZCU IS CURRENTLY WORKING ON THIS"

subway['cbg_income'] = [0]*92796
for row in cosa.iterrows():
    #print(row[1][0], row[1][1])
    subway['cbg_income'] = np.where(str(subway['cbg_income']) == str(row[1][0]), row[1][1], subway['cbg_income'])


In [55]:
subway['cbg_income']

3441     0.0
3442     0.0
3443     0.0
3444     0.0
3445     0.0
        ... 
90590    0.0
90591    0.0
90592    0.0
90593    0.0
90594    0.0
Name: cbg_income, Length: 92796, dtype: float64